<a href="https://colab.research.google.com/github/ademiltonnunes/Machine-Learning/blob/main/ChatGPT/Fine-Tuning/2000%20Drug%20Examples/Advanced_Fine_Tuning_Drug_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Preparing the Data and Launching the Fine Tuning**

##Instalation


In [2]:
! pip install pandas openpyxl openai==0.28

##Import

In [19]:
# Use Pandas to transform the data into the desired format.
import pandas as pd

##Preparing the Data and Launching the Fine Tuning

In [20]:
######################################################################
# Read the first n rows from the Excel file
# - The number of rows to read from the Excel file,
#   Medicine_description.xlsx, to 2000.
#   + This means that we are going to use a dataset of 2000 drug
#     names to fine-tune the model.
# - You can use more.
######################################################################
n = 2000

######################################################################
# Kaggle data
# - Company_Name.xlsx
# - Medicine_description.xlsx - 3 columns
#   + Drug_Name
#   + Reason
#   + Description
# - Ratings.xlsx
######################################################################

# Reading the first n rows of data from the Excel file
# 'Medicine_description.xlsx’ and stores it in a data frame called df.
n = 2000
df = pd.read_excel('Medicine_description.xlsx', sheet_name='Sheet1', header=0, nrows=n)
df.head()

,Drug_Name,Reason,Description
0,A CN Gel(Topical) 20gmA CN Soap 75gm,Acne,Mild to moderate acne (spots)
1,A Ret 0.05% Gel 20gmA Ret 0.1% Gel 20gmA Ret 0...,Acne,A RET 0.025% is a prescription medicine that i...
2,ACGEL CL NANO Gel 15gm,Acne,It is used to treat acne vulgaris in people 12...
3,ACGEL NANO Gel 15gm,Acne,It is used to treat acne vulgaris in people 12...
4,Acleen 1% Lotion 25ml,Acne,treat the most severe form of acne (nodular ac...


In [21]:
# Get the unique values in the ‘Reason’ column of the data frame,
# stores them in an array called reasons
reasons = df["Reason"].unique()
print(reasons)

['Acne' 'Adhd' 'Allergies' 'Alzheimer' 'Amoebiasis' 'Anaemia' 'Angina']


In [22]:
# Assigns a numerical index to each unique value in the reasons
# array, and stores it in a dictionary called reasons_dict.
reasons_dict = {reason: i for i, reason in enumerate(reasons)}
print(reasons_dict)

{'Acne': 0, 'Adhd': 1, 'Allergies': 2, 'Alzheimer': 3, 'Amoebiasis': 4, 'Anaemia': 5, 'Angina': 6}


In [23]:
# Add a new line and “Malady:” to the end of each drug name in
# the ‘Drug_Name’ column of the data frame.
# - The desired format:
#       Drug: <Drug_Name>\nMalady:
df["Drug_Name"] = "Drug: " + df["Drug_Name"] + "\n" + "Malady:"
df.head()

,Drug_Name,Reason,Description
0,Drug: A CN Gel(Topical) 20gmA CN Soap 75gm\nMa...,Acne,Mild to moderate acne (spots)
1,Drug: A Ret 0.05% Gel 20gmA Ret 0.1% Gel 20gmA...,Acne,A RET 0.025% is a prescription medicine that i...
2,Drug: ACGEL CL NANO Gel 15gm\nMalady:,Acne,It is used to treat acne vulgaris in people 12...
3,Drug: ACGEL NANO Gel 15gm\nMalady:,Acne,It is used to treat acne vulgaris in people 12...
4,Drug: Acleen 1% Lotion 25ml\nMalady:,Acne,treat the most severe form of acne (nodular ac...


In [24]:
# It concatenates a space and the corresponding numerical index
# from the reasons_dict to the end of each 'Reason’
# value in the data frame.
df["Reason"] = " " + df["Reason"].apply(lambda x: "" + str(reasons_dict[x]))


In [25]:
df.head()

,Drug_Name,Reason,Description
0,Drug: A CN Gel(Topical) 20gmA CN Soap 75gm\nMa...,0,Mild to moderate acne (spots)
1,Drug: A Ret 0.05% Gel 20gmA Ret 0.1% Gel 20gmA...,0,A RET 0.025% is a prescription medicine that i...
2,Drug: ACGEL CL NANO Gel 15gm\nMalady:,0,It is used to treat acne vulgaris in people 12...
3,Drug: ACGEL NANO Gel 15gm\nMalady:,0,It is used to treat acne vulgaris in people 12...
4,Drug: Acleen 1% Lotion 25ml\nMalady:,0,treat the most severe form of acne (nodular ac...


In [28]:
# For this example, we don’t need the ‘Description’ column, that’s
# why the script drops it from the data frame.
df.drop(["Description"], axis=1, inplace=True)

In [32]:
df.head()

,prompt,completion
0,Drug: A CN Gel(Topical) 20gmA CN Soap 75gm\nMa...,0
1,Drug: A Ret 0.05% Gel 20gmA Ret 0.1% Gel 20gmA...,0
2,Drug: ACGEL CL NANO Gel 15gm\nMalady:,0
3,Drug: ACGEL NANO Gel 15gm\nMalady:,0
4,Drug: Acleen 1% Lotion 25ml\nMalady:,0


In [30]:
# Renaming the ‘Drug_Name’ column to ‘prompt’
# and the ‘Reason’ column to ‘completion’.
df.rename(columns={"Drug_Name": "prompt", "Reason": "completion"}, inplace=True)

In [31]:
df.head()

,prompt,completion
0,Drug: A CN Gel(Topical) 20gmA CN Soap 75gm\nMa...,0
1,Drug: A Ret 0.05% Gel 20gmA Ret 0.1% Gel 20gmA...,0
2,Drug: ACGEL CL NANO Gel 15gm\nMalady:,0
3,Drug: ACGEL NANO Gel 15gm\nMalady:,0
4,Drug: Acleen 1% Lotion 25ml\nMalady:,0


In [14]:
# Convert the dataframe to jsonl format
jsonl = df.to_json(orient="records", indent=0, lines=True)
with open("drug_malady_data.jsonl", "w") as f:
    f.write(jsonl)

In [15]:
jsonl

'{"prompt":"Drug: A CN Gel(Topical) 20gmA CN Soap 75gm\\nMalady:","completion":" 0"}\n{"prompt":"Drug: A Ret 0.05% Gel 20gmA Ret 0.1% Gel 20gmA Ret 0.025% Gel 20gm\\nMalady:","completion":" 0"}\n{"prompt":"Drug: ACGEL CL NANO Gel 15gm\\nMalady:","completion":" 0"}\n{"prompt":"Drug: ACGEL NANO Gel 15gm\\nMalady:","completion":" 0"}\n{"prompt":"Drug: Acleen 1% Lotion 25ml\\nMalady:","completion":" 0"}\n{"prompt":"Drug: Aclene 0.10% Gel 15gm\\nMalady:","completion":" 0"}\n{"prompt":"Drug: Acnay Gel 10gm\\nMalady:","completion":" 0"}\n{"prompt":"Drug: Acne Aid Bar 50gmAcne Aid Bar 100gm\\nMalady:","completion":" 0"}\n{"prompt":"Drug: Acne UV Gel 60gm\\nMalady:","completion":" 0"}\n{"prompt":"Drug: Acne UV SPF 30 Gel 30gm\\nMalady:","completion":" 0"}\n{"prompt":"Drug: Acnecure Gel 20gm\\nMalady:","completion":" 0"}\n{"prompt":"Drug: Acnedap Gel 15gm\\nMalady:","completion":" 0"}\n{"prompt":"Drug: Acnedap Plus Gel 15gm\\nMalady:","completion":" 0"}\n{"prompt":"Drug: Acnehit Gel 15gm\\nMalad

In [16]:
# Write the jsonl to a file
#
# - drug_malady_data.jsonl has data like
#    [..]
#    {"prompt":"Drug: Acleen 1% Lotion 25ml\nMalady:","completion":" 0"}
#    [..]
#    {"prompt":"Drug: Capnea Injection 1ml\nMalady:","completion":" 1"}
#    [..]
#    {"prompt":"Drug: Mondeslor Tablet 10'S\nMalady:","completion":" 2"}
#    [..]
with open("drug_malady_data.jsonl", "w") as f:
    f.write(jsonl)